In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from fractions import Fraction
from decimal import *
TWOPLACES = Decimal(10) ** -2
from natsort import natsorted
import ast
import time
today = time.strftime("%Y_%m_%d")
from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float,reg_ex,clean_data     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [2]:
# parameters
customer_id = '5'
formatted_attribute = 'rug_size'
customer_name='%bedbathandbeyond%'
buckets = """Office Chairs"""

attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [3]:
import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
fields = ['LONG_DESCRIPTION']
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']

# reg_ex(df)
print('Length of Total Dataframe: '+str(len(df)))
df.drop_duplicates(subset='external_id', inplace=True)
df=df[(df['external_id'].astype(str)!='200075904')&(df['external_id'].astype(str)!='200045715')&(df['external_id'].astype(str)!='200045714')&(df['external_id'].astype(str)!='200046141')&(df['external_id'].astype(str)!='200046271')]
import collections
print('REPEATING External IDs: '+str([item for item, count in collections.Counter(df['external_id']).items() if count > 1]))



del df['long_desc']

Length of Total Dataframe: 2829
REPEATING External IDs: []


In [4]:
power=r'''(?i)(13'. Runner)|(14' runner)|(15' runner)|(16' runner)|(16'5" runner)|(17' runner)|(18' runner)|(19' runner)|(19'8" runner)|(2'.?[67]"? x 1[3-9](?:\.\d)?')|(2' 6" x 16')|(2' 6" x 18')|(2' 8" x 20')|(2' x 16' runner)|(2' x 20' runner)|(20' runner)|(21' runner)|(22' runner)|(2'2" x 13' runner)|(2'2" x 14' runner)|(2'3" x 14' runner)|(2'3" x 16' runner)|(2'3" x 18' runner)|(2'3" x 20' runner)|(2'3" x 22' runner)|(2'6 x 14' runner)|(2'6" x 14' runner)|(2'6" x 16' runner)|(2'6" x 18' runner)|(2'6" x 20' runner)|(2'6" x 22' runner)|(2'7" x 14' runner)|(2'7" x 16' runner)|(2'7" x 20' runner)|(30'lx20"wx0\.25"h)|(runner 2' 6" x 14')|(runner 2' 8" x 14')|(runner 2' 8" x 16')|(runner 2' 8" x 18')|(runner 2' 8" x 20')|()'''
df['13Runner'] = df['product_name'].apply(lambda x: re_extract(power, str(x)))

power=r'''(?i)(12' Runner)|(11'10" runner)|(11'7" runner)|(11'8" runner)|(11'9" runner)|(12' runner)|(13' runner)|(2' 6" x 12')|(2' 8" x 12')|((?<!\d)2 x 12)|((?<!\d)2' x 12')|(2'2" x 12')|(2'2" x 12' runner)|(2'3 x 11'9 runner)|(2'3" x 11'9")|(2'3" x 12')|(2'3" x 12' runner)|(2'3"x12')|(2'5" x 13' runner)|(2'6 x 12' runner)|(2'6" x 11'9")|(2'6" x 12' runner)|(2'7" x 12')|(2'8" x 12' runner)|(3' x 12' runner)|(runner 2' 6" x 12')|(runner 2' 8" x 12')|(runner 2' x 12')|()'''
df['12Runner'] = df['product_name'].apply(lambda x: re_extract(power, str(x)))

power=r'''(?i)(11' Runner)|(10'6" runner)|(11' runner)|(2'2" x 10'9" runner)|(2'2" x 11' runner)|(2'3" x 11' runner)|(2'5" x 10'6" runner)|()'''
df['11Runner'] = df['product_name'].apply(lambda x: re_extract(power, str(x)))

power=r'''(?i)(10' Runner)|(10' runner)|(10'3" runner)|(2' 4" x 9' 6")|(2' 5" x 9' 5")|(2'.?6"? x 10')|(2' 8" x 10')|((?<!')2 x 10)|(2' x 10')|(2' x 10' runner)|(2'2" x 10')|(2'2" x 10' runner)|(2'2" x 9'10")|(2'3" x 10')|(2'3" x 10' runner)|(2'3" x 10'2")|(2'3" x 9'10" runner)|(2'3"x10')|(2'6" x 10')|(2'6" x 10' runner)|(2'6" x 9'6" runner)|(2'7" x 10')|(2'7" x 10' runner)|(2'8" x 10' runner)|(2'8" x 10'2" runner)|(2'8" x 9'10")|(3' x 10' runner)|(3'3" x 10' runner)|(4' x 10')|(9'10" runner)|(9'5" runner)|(9'6" runner)|(9'9" runner)|(runner 2' 5" x 9' 6")|(runner 2' 6" x 10')|(runner 2' 6" x 9' 6")|(runner 2' 8" x 10')|(runner 2' x 10')|(2'[67] x 10')|()'''
df['10Runner'] = df['product_name'].apply(lambda x: re_extract(power, str(x)))

power=r'''(?i)(9' Runner)|(2' 3" x 8' 5")|(2'2" x 9' runner)|(2'3" x 9' runner)|(24" x 108")|(30" x 108")|(8'11" runner)|(8'5" runner)|(8'6" runner)|(8'9" runner)|(9' runner)|(9'1" runner)|()'''
df['9Runner'] = df['product_name'].apply(lambda x: re_extract(power, str(x)))

power=r'''(?i)((?<!)8' Runner)|(1'11" x 7'6")|(1'11" x 7'6" runner)|(1'8" x 7'6")|(2' 3" x 7' 10")|(2' 5" x 8')|(2' 6" x 8')|(2' 8" x 8')|(2' x 7'10")|(2 x 8)|(2' x 8')|(2' x 8' runner)|(2'1" x 7'8")|(2'1" x 8" runner)|(2'10" x 7'10" runner)|(2'2 x 7'6)|(2'2 x 7'7)|(2'2 x 7'9)|(2'2" x 7'10" runner)|(2'2" x 7'7")|(2'2" x 7'7" runner)|(2'2" x 8' runner)|(2'2" x 8'1")|(2'2"x 7'6")|(2'2"x7'6")|(2'3' x 7'6")|(2'3 x 8')|(2'3" x  8')|(2'3" x 7'10")|(2'3" x 7'10" runner)|(2'3" x 7'5" runner)|(2'3" x 7'6")|(2'3" x 8')|(2'3" x 8')|(2'3" x 8' runner)|(2'3"x 8')|(2'3"x7'6")|(2'4 x 8')|(2'4" x 7'10")|(2'4" x 8')|(2'4" x 8' runner)|(2'4" x 8'2")|(2'4" x 8'6")|(24" x 96")|(2'5" x 7'7" runner)|(2'6" x 8' runner)|(2'6" x 7'6" runner)|(2'6" x 8')|(2'6" x 8' runner)|(2'7" x 7'10" runner)|(2'7" x 7'6")|(2'7" x 7'9" runner)|(2'7" x 8' runner)|(2'7" x 8'2" runner)|(27"x8\.25")|(2'8" x 8' runner)|(2'8" x 8'2")|(3' x 8' runner)|(7'10 runner)|(7'10" runner)|(7'11" runner)|(7'5" runner)|(7'6" runner)|(7'7" runner)|(7'9" runner)|(7'9" x 7'9")|((?<!\d)8' runner)|(8'1" runner)|(8'2" runner)|(runner 2' 2" x 8')|(runner 2' 5" x 8')|(runner 2' 6" x 8')|(runner 2' 8" x 8')|(runner 2' x 8')|(2'[3-7].?"? x 8')|()'''
df['8Runner'] = df['product_name'].apply(lambda x: re_extract(power, str(x)))

power=r'''(?i)(7' Runner)|(1'10" x 7'2" runner)|(1'9" x 6'9")|(1'9" x 7' runner)|((?<!\d')2' x 7')|(2' x 7' runner)|(2'1" x 7')|(2'2 x 7'3)|(2'2" x 7')|(2'2" x 7' runner)|(2'2" x 7'2" runner)|(2'2" x 7'3")|(2'2" x 7'3" runner)|(2'[23].?" x 7'6.?")|(2'2" x 7'6" runner)|(2'3" x 7' runner)|(2'3" x 7'3")|(2'3" x 7'3" runner)|(2'3" x 7'6" runner)|(24 in l x 78 in w)|(2'6" x 6'6")|(2'6" x 7' runner)|(2'7" x 7'2" runner)|(2'7" x 7'4" runner)|(2'7" x 7'6" runner)|(2'8" x 7'6")|(32 in l x 78 in w)|(3'3" x 6'7")|(6'10" runner)|(6'11" runner)|(6'5" runner)|(6'7" runner)|(6'8" x 3'3")|(7' runner)|(7'1" runner)|(7'2" runner)|(7'3" runner)|(7'4" runner)|(86"d x 31"w x 0\.39"h)|(86"d x 63"w x 0\.39"h)|(86"l x 31"w x 0\.79"h)|(86"l x 63"w x 0\.79"h)|(88"l x 31"w x 0\.53"h)|()'''
df['7Runner'] = df['product_name'].apply(lambda x: re_extract(power, str(x)))

power=r'''(?i)((?<!\d)6' Runner)|(1'10" x 6'(?! \d"))|(1'10" x 6' runner)|(1'10"x6'(?! \d"))|(1'8" x 6'3")|(2' 6" x 6')|(2' 8" x 6'(?! \d"))|(2' x 5'9")|(2' x 6'(?! \d"))|(2' x 6' runner)|(2\.5' x 6'(?! \d"))|(20" x 72")|(2'2"? x 6'(?! \d"))|(2'2" x 6' runner)|(2'2" x 6'2" runner)|(2'2" x 6'7" runner)|(22" x 72")|(2'3" x 6'(?! \d"))|(2'3" x 6' runner)|(24 in l x 70 in w)|(24" x 66")|(24" x 68")|(24" x 72")|(2'6 x 6' runner)|(2'6" x 6'(?! \d"))|(2'6" x 6' runner)|(2'7" x 6' runner)|(2'x5'9")|(2'x6'(?! \d"))|(30" x 72")|(32 in l x 70 in w)|(3'4" x 5'6")|(36" x 72")|(5'10" runner)|(5'6" runner)|(5'7" runner)|(5'9" runner)|((?<!\d)6' runner)|(6'2" runner)|(6'3" runner)|(72\.04x24\.01x0\.39)|(72\.05x24\.02x1\.96)|(72x24\.01x0\.39)|(runner 2' 6" x 6'(?! \d"))|(runner 2' 8" x 6'(?! \d"))|(runner 2' x 6'(?! \d"))|()'''
df['6Runner'] = df['product_name'].apply(lambda x: re_extract(power, str(x)))

power=r'''(?i)((?<!\d)5' Runner)|(1'10" x 4'6")|(1'10" x 5')|(1'10" x 5'1")|(1'7" x 4'10")|(1'8" x 4'6")|(1'8" x 5')|(1'8" x 5' runner)|(18" x 54")|(1'9" x 4'9")|(2 x 5)|(2' x 5')|(2' x 5' runner)|(20" x 66")|(21" x 60")|(2'2" x 5' runner)|(22" x 60")|(2'3" x 5')|(23" x 60")|(24 in l x 60 in w)|(2'4" x 4'7")|(24" x 58")|(24" x 60")|(25" x 60")|(4'11" runner)|(4'6" runner)|((?<!\d)5' runner)|(59" x 20")|(59" x 24")|(60" runner)|(60" x 24")|()'''
df['5Runner'] = df['product_name'].apply(lambda x: re_extract(power, str(x)))

power=r'''(?i)((?<!\d)4' Runner)|(1'4" x 3'11")|(16 in l x 48 in w)|(1'8" x 3'9")|(1'9" x 3'9")|(2' x 4')|(20 in l x 48 in w)|(20" x 55")|(2'2 x 4'3)|(2'2" x 3'7")|(2'2" x 4')|(2'2" x 4'3")|(22" x 47")|(22" x 52")|(2'3" x 4'5")|(2'3" x 4'6")|(24 in l x 48 in w)|(24" x 48")|((?<!\d)4' runner)|(2'6.?" x 4')|()'''
df['4Runner'] = df['product_name'].apply(lambda x: re_extract(power, str(x)))

power=r'''(?i)(11'. Round\/Square)|(11' round)|(11'9" round)|(11'9" x 11'9")|(12' round)|(12' x 12')|(13'10" x 13'10")|(13'10" x 13'11")|()'''
df['11Round'] = df['product_name'].apply(lambda x: re_extract(power, str(x)))

power=r'''(?i)(10'. Round\/ square)|(10' round)|(10' square)|(10' x 10')|(9'10" round)|(9'6" round)|(9'9" round)|(9'9" x 9'9")|(square 10')|()'''
df['10Round'] = df['product_name'].apply(lambda x: re_extract(power, str(x)))

power=r'''(?i)(9' Round\/ square)|(8'6" round)|(8'6" x 8'6")|(8'6" x square)|(8'9" round)|(8'9" x 8'9")|((?<!\d' x )9' oval)|(9' round)|(9' square)|(9' x 9')|(9' x square)|(9' x square)|(9'2" round)|(9'3" round)|()'''
df['9Round'] = df['product_name'].apply(lambda x: re_extract(power, str(x)))

power=r'''(?i)(8' Round\/ square)|(7' 10" round)|(7' 10" square)|(7' 6" round)|(7' 6" square)|(7'10 x round)|(7'10" round)|(7'10" square)|(7'10" x  squ)|(7'10" x 7'10")|(7'10" x 8')|(7'10" x round)|(7'10" x squ)|(7'10" x square)|(7'10" xrnd)|(7'10"x squ)|(7'5" round)|(7'6" round)|(7'6" x 7'6")|(7'6" x round)|(7'7" octagon)|(7'7" round)|(7'7" square)|(7'7" x 7'7")|(7'9" round)|(7'9" x round)|(7'9" x square)|(8' octagon)|(8'\d? round)|(8' square)|(8' x 8')|(8' x round)|(8' x square)|(8" l x 8" w)|(8' x 8')|(8'2" round)|(8'2" x 8'2")|(8'2" x round)|(8'xround)|(94"l x 94"w x 0\.53"h)|(96" round)|(96" x 96")|(round 8')|(square 7' 10")|(square 7' 6")|(square 8')|(8'\d? (?:Square|Sqaure))|()'''
df['8Round'] = df['product_name'].apply(lambda x: re_extract(power, str(x)))

power=r'''(?i)(7' Round\/ square)|(6'10" round)|(6'6" round)|(6'6" x 6'6")|(6'6" x square)|(6'7" round)|(6'7" square)|(6'7" x 6'7")|(6'7" x 6'7" round)|(6'7" x round)|(6'7" x square)|(6'8" round)|(7' round)|(7' square)|(7' x 7')|(7' x 7'4")|(7' x square)|(7'2" round)|(7'2" x 7'2")|(7'3" round)|(7'3" x 7'3")|(round 6' 7")|(square 6' 7")|()'''
df['7Round'] = df['product_name'].apply(lambda x: re_extract(power, str(x)))

power=r'''(?i)(6' Round\/ square)|(5'5" round)|(5'6" round)|(5'6" x 5'6")|(5'6" x round)|(5'7" round)|(5'8" x 6')|(5'9" round)|(5'9" x 5'9")|(6' octagon)|(6' round)|((?<!4' x )6' square)|((?<!\d')6 x 6)|(6' x 6')|(6' x 6' 11")|(6' x 6'round)|(6' x round)|(6" x 6")|(6'3" x 6'3")|(6'[4-7]"? round)|(6'4" x 6'4")|(72" x 72")|(round 5' 5")|(round 6')|(round 6' round)|(round 6'x6')|(square 6')|(square 6'x6')|()'''
df['6Round'] = df['product_name'].apply(lambda x: re_extract(power, str(x)))

power=r'''(?i)(5' Round\/ square)|(4'10" round)|(5' 3" round)|(5' octagon)|((?<!3' x )5' oval)|(5' round)|(5' square)|(5' x 5')|(5' x round)|(5' x square)|(5"x5")|(5'1" square)|(5'1" round)|(5'1" x 5'1")|(5'2" round)|(5'3 x round)|(5'3" rnd)|(5'3" round)|(5'3" square)|(5'3" x 5'3")|(5'3" x 5'3"square)|(5'3" x round)|(5'3" x square)|(5'3" xrnd)|(5'3"x round)|(5'3"x5'3" square)|(5'3"xround)|(5'xround)|(60" round)|(60" x 60")|(round 5')|(round 5' 3")|(square 5'(?!\d x))|()'''
df['5Round'] = df['product_name'].apply(lambda x: re_extract(power, str(x)))

power=r'''(?i)(4' Round\/ square)|(3' 6" x round)|(3'10" x round)|(3'10" xrnd)|(3'11" round)|(3'11" x round)|(3'6" round)|(3'6" x 3'6")|(3'9" round)|(3'9" x 3'9")|(4' rectangle)|(4' round)|(4' square)|(4' x 4')|(4' x round)|(4" x 4")|(4'3" x 4'3")|(4'4" square)|(45" x 53")|(48" x 48")|(48"w x 48"d x 0\.375"h)|(4'x round)|(4'x4')|(4'xround)|(51" x 48")|(53" x 48")|(round 4')|(square 4')|()'''
df['4Round'] = df['product_name'].apply(lambda x: re_extract(power, str(x)))

power=r'''(?i)(3' Round\/ square)|(2'3" round)|(2'7" round)|(2'7" x 2'7")|(2'8" round)|(2'9" x 3'3")|(3' round)|(3' x 3')|(3' x 3\.5')|(30" round)|(30" x 34")|(3'11" x 3'11")|(3'3" round)|(3'3" x 3'3")|(3'[1-4]"? round)|(3'4" x 3'4")|(3'4" x round)|(35" round)|(36" round)|(38" x 39")|(39" x 39")|()'''
df['3Round'] = df['product_name'].apply(lambda x: re_extract(power, str(x)))

power=r'''(?i)(12' x 15')|(11' x 15')|(11' x 17')|(11'1" x 15'8")|(11'10" x 14'10")|(11'6" x 15')|(11'6" x 15'6")|(11'6" x 15'7")|(11'8" x 14'8")|(12' 8" x 15')|(12' x 14')|(12' x 14'5")|(12' x 15')|(12' x 16')|(12' x 18')|(12\.6' x 15\.35')|(12'10" x 15'6")|(12'6" x 15")|(13' x 15')|(13' x 18')|()'''
df['12x15'] = df['product_name'].apply(lambda x: re_extract(power, str(x)))

power=r'''(?i)(10' x 14')|(10' 2" x 14')|(10' 6" x 14')|(10' x 13')|(10' x 13'2")|(10' x 13'6")|(10' x 14')|(10'2" x 13'9")|(10'2" x 14')|(10'2" x 14'1")|(10'3" x 14')|(10'3" x 14')|(10'3" x 14'3")|(10'5" x 14')|(10'6" x 14')|(10'9" x 13'2")|(11' x 14')|(11' x 14' 6")|(11'4" x 14'4")|(9' 10" x 13' 8")|(9' 10" x 14')|(9' 6" x 13' 6")|(9'10" x 13'10")|(9'10" x 13'9")|(9'10" x 14')|(9'11" x 13'9")|(9'11" x 14')|(9'6" x 12'8")|(9'6" x 13'6")|(9'6" x 13'9")|(9'9" x 13'9")|()'''
df['10x14'] = df['product_name'].apply(lambda x: re_extract(power, str(x)))

power=r'''(?i)(9' x 12')|(10" x 12")|(8' 10" x 12')|(8' 4" x 12')|(8' 6" x 11')|(8' 6" x 11' 6")|(8' 6" x 12')|(8' 6" x 12' 2")|(8' 6" x 13')|(8'10 x 11'10)|(8'10" x 11')|(8'10" x 11'10")|(8'10" x 11'9")|(8'10" x 12')|(8'10" x 12'2")|(8'10" x 12'3")|(8'10" x 12'7")|(8'10" x 12'9")|(8'10" x 13')|(8'11" x 11'11")|(8'2" x 12'5")|(8'6" x 11'10")|(8'6" x 11'6")|(8'6" x 11'7")|(8'6" x 12')|(8'6" x 12'11")|(8'6" x 12'6")|(8'6" x 12'9")|(8'6" x 13')|(8'6" x 13'")|(8'7" x 11'6")|(8'7" x 12')|(8'8" x 11'10")|(8'8" x 12')|(8'9" x 11'10")|(8'9" x 11'11")|(8'9" x 11'8")|(8'9" x 11'9")|(8'9" x 12')|(8'9" x 12'2")|(8'9" x 12'5")|(8'9" x 12'9")|(8'9" x 13')|(9' 10" x 13')|(9' 2" x 12')|(9' 6" x 12')|(9' x 11')|(9' x 11'7")|(9' x 11'9")|(9 x 12)|(9' x 12')|(9' x 12' 6")|(9' x 12'10")|(9' x 12'3")|(9' x 12'6")|(9' x 12'7")|(9' x 13')|(9' x 13'1")|(9' x 13'3")|(9' x 13'6")|(9'0" x 12'2")|(9'1" x 12'9")|(9'10" x 12'10")|(9'10" x 12'6")|(9'10" x 12'6")|(9'10" x 13')|(9'10" x 13'1")|(9'10" x 13'11")|(9'10" x 13'2")|(9'10"x13'2")|(9'2" x 12')|(9'2" x 12")|(9'2" x 12'1")|(9'2" x 12'2")|(9'2" x 12'3")|(9'2" x 12'4")|(9'2" x 12'5")|(9'2" x 12'7")|(9'2" x 12'9")|(9'2" x 13')|(9'2" x 13'1")|(9'2" x 13'2")|(9'3 x 12'9)|(9'3" x 11'10")|(9'3" x 12')|(9'3" x 12'1")|(9'3" x 12'2")|(9'3" x 12'3")|(9'3" x 12'5")|(9'3" x 12'6")|(9'3" x 12'9")|(9'3" x 13')|(9'3" x 13'0")|(9'3" x 13'3")|(9'4" x 12'6")|(9'4" x 13'2")|(9'4" x 13'4")|(9'6" x 12')|(9'6" x 12'11")|(9'6" x 12'6")|(9'6" x 12'7")|(9'6" x 13')|(9'6" x 13'1")|(9'6" x 13'2")|(9'6" x 13'3")|(9'6" x 13'4")|(9'8" x 12'7")|(9'9" x 12'2")|(9'9" x 12'6")|(9'9" x 13')|(9'x12')|()'''
df['9x12'] = df['product_name'].apply(lambda x: re_extract(power, str(x)))

power=r'''(?i)(8' x 10')|(10' x 8')|(120\.08x96\.06x1\.96)|(120x96x0\.39)|(122\.04x94\.48x0\.31)|(126"l x 94"w x 0\.53"h)|(7' 10" x 10')|(7' 10" x 10' 10")|(7' 10" x 11')|(7' 10" x 11' 2")|(7' 10" x 9' 10")|(7' 10" x 9' 8")|(7' 6" x 10')|(7' 6" x 10' 9")|(7' 6" x 9' 10")|(7' x 10')|(7'10 x 10')|(7'10 x 10'10)|(7'10 x 10'6)|(7'10 x 10'9)|(7'10 x 9'10)|(7'10" x  9'10")|(7'10" x 10')|(7'10" x 10'1")|(7'10" x 10'10")|(7'10" x 10'2")|(7'10" x 10'3")|(7'10" x 10'4")|(7'10" x 10'5")|(7'10" x 10'6")|(7'10" x 10'7")|(7'10" x 10'8")|(7'10" x 10'9")|(7'10" x 11')|(7'10" x 11'1")|(7'10" x 11'2")|(7'10" x 9'10")|(7'10" x 9'10")|(7'10" x 9'11")|(7'10" x 9'8")|(7'10" x 9'9")|(7'10"x10'6")|(7'10"x9'10")|(7'11" x 10')|(7'11" x 10'10")|(7'11" x 9'10")|(7'4" x 10'6")|(7'6' x 10')|(7'6" x 10')|(7'6" x 10'1")|(7'6" x 10'2")|(7'6" x 10'3")|(7'6" x 10'6")|(7'6" x 10'8")|(7'6" x 10'9")|(7'6" x 11')|(7'6" x 9'5")|(7'6" x 9'6")|(7'6" x 9'7")|(7'6"? x 9'[86]"?)|(7'7" x 10')|(7'7" x 10'10")|(7'7" x 10'2")|(7'7" x 10'5")|(7'7" x 10'6")|(7'7" x 9'6")|(7'8" x 10')|(7'8" x 10'1")|(7'8" x 10'10")|(7'8" x 10'11")|(7'8" x 10'11")|(7'8" x 10'9")|(7'9 x 9'9)|(7'9" x 10')|(7'9" x 10'10")|(7'9" x 10'2")|(7'9" x 10'5")|(7'9" x 10'6")|(7'9" x 11')|(7'9" x 11'2")|(7'9" x 9'10")|(7'9" x 9'5")|(7'9" x 9'6")|(7'9" x 9'7")|(7'9" x 9'9")|(7'9"x10'10")|(7'9"x9'9")|(8' 10" x 9' 8")|(8' 2" x 10')|(8' 2" x 11' 6")|(8' 3" x 11')|(8' 4" x 11' 6")|(8' 6" x 10' 6")|(8' x  10')|((?<!')8 x 10)|(8' x 10')|(8' x 10')|(8' x 10' oval)|(8' x 10'1")|(8' x 10'2")|(8' x 10'3")|(8' x 10'4")|(8' x 10'5)|(8' x 10'5")|(8' x 10'6)|(8' x 10'6")|(8' x 10'9")|(8' x 11')|(8' x 11'2")|(8' x 11'4")|(8' x 9' 8")|(8" x 11")|(8'1" x 10')|(8'1" x 10'2")|(8'1" x 11'2")|(8'1" x 11'3")|(8'2 x 10')|(8'2" x 11'6")|(8'2" x 11'8")|(8'3' x 11'6")|(8'3" x 10')|(8'3" x 11')|(8'3" x 11")|(8'3" x 11'6")|(8'3'x11'6")|(8'4" x 10'2")|(8'4" x 11'6")|(8'4" x 11'7")|(8'6" x 11')|(8'7" x 10'6")|(8'x 10')|(8'x10'6")|(96" x 132")|(oval 8' x 10')|()'''
df['8x10'] = df['product_name'].apply(lambda x: re_extract(power, str(x)))

power=r'''(?i)(7' x 9')|(6' 7" x 8' 6")|(6' 7" x 9')|(6' 7" x 9' 4")|(6' 7" x 9' 6")|(6' 7" x 9' 8")|(6'5" x 8'8")|(6'5" x 9'8")|(6'6 x 9'6)|(6'6" x 10')|(6'6" x 9')|(6'6" x 9'10")|(6'6" x 9'3")|(6'6" x 9'4")|(6'6" x 9'5")|(6'6" x 9'8")|(6'6"x9'6")|(6'7 x 9'6)|(6'7 x 9'7)|(6'7" x  9'6")|(6'7" x 8'2")|(6'7" x 9')|(6'7" x 9'1")|(6'7" x 9'2")|(6'7" x 9'3")|(6'7" x 9'4")|(6'7" x 9'6")|(6'7" x 9'7")|(6'7" x 9'8")|(6'7"x9'6")|(6'8" x 10')|(6'9" x 9'6")|(7' 6" x 9' 6")|(7' x 9')|(7' x10')|(7'1" x 10')|(7'2" x 10'3")|(7'2" x 10'5")|(7'[1-3]"? x 10'[1-6]"?)|(7'3" x 9'2")|(7'3" x 9'3")|(7'4" x 10'3")|(7'4" x 9'6")|(7'6" x 9')|(7'6" x 9'3")|(7'6" x 9'6" oval)|(88" x 112")|(96" x 120")|(oval 6' 7" x 9')|(oval 7' 6" x 9' 6")|(7'3.?" x 9'3.?")|()'''
df['7x9'] = df['product_name'].apply(lambda x: re_extract(power, str(x)))

power=r'''(?i)(6' x 9')|(1"dx8 ft"wx6 ft"h)|(5'10" x 9'2")|(5'11" x 9')|(5'11" x 9'1")|(5'11" x 9'2")|(5'7" x 8'6")|(5'8" x 8'8")|(5'9" x 9')|(5'9" x 9'2")|(6' 3" x 9')|(6' 3" x 9' 2")|(6'.?[67]"? x [98]')|(6' 7" x 8' 2")|(6' 7" x 9' oval)|((?<!\.)6' x 8')|(6' x 8'8")|((?<!\d')6 x 9)|(6' x 9')|(6' x 9' 6")|(6' x 9' oval)|(6' x 9'6")|(6'2" x 8')|(6'3" x 9'2")|(6'3" x 9'6")|(6'4" x 8')|(6'6" x 9'2")|(6'6" x 9'6")|(6'7" x 9'10")|(6'x9')|(72" x 108")|(oval 6' x 9')|()'''
df['6x9'] = df['product_name'].apply(lambda x: re_extract(power, str(x)))

power=r'''(?i)(5' x 8')|(1"dx8 ft"wx4 ft"h)|(1"dx8 ft"wx6 ft"h)|(1\.5"dx8 ft"wx4 ft"h)|(4' x 8')|(4'10" x 7'6")|(4'11" x 7'10")|(4'11" x 7'6")|(4'6" x 8')|(4'9" x 7'10")|(4'9" x 7'6")|(5' 3" x 7' 6")|(5' 3" x 7' 7")|(5' 3" x 7' 8")|(5' 3" x 8')|(5' 5" x 8')|(5' 6" x 8' 3")|(5' 9" x 7' 7")|(5' x 7' 5")|(5' x 7' 6")|(5' x 7' 7")|(5' x 7' 9")|(5' x 7'10")|(5' x 7'5")|(5' x 7'6)|(5' x 7'6")|(5' x 7'7")|(5' x 7'[789]"?)|(5' x 7'9")|(5 x 8)|(5' x 8')|(5' x 8' oval)|(5' x 8'1")|(5' x 8'2")|(5'0" x 7'6")|(5'1" x 7'5")|(5'1" x 7'6")|(5'1" x 7'7")|(5'1" x 7'8")|(5'1" x 8')|(5'2" x 7'5")|(5'2" x 7'6")|(5'2" x 7'7")|(5'2" x 7'8")|(5'3 x 7'10)|(5'3 x 7'5)|(5'3 x 7'6)|(5'3 x 7'7)|(5'3" x 7'10")|(5'3" x 7'11")|(5'3" x 7'6")|(5'3" x 7'6" oval)|(5'3" x 7'7")|(5'3" x 7'8")|(5'3" x 7'9")|(5'3" x 8')|(5'3" x 8'3")|(5'3"x7'5")|(5'3"x7'6")|(5'4" x 7'8")|(5'4" x 8')|(5'5" x 7'11")|(5'5" x 7'6")|(5'5" x 7'7")|(5'5" x 7'9")|(5'5" x 8')|(5'5" x 8'4")|(5'6 x 8')|(5'6" x 7'5")|(5'6" x 7'6")|(5'6" x 7'9")|(5'6" x 8')|(5'6" x 8'6")|(5'6"x8')|(5'7" x 7'6")|(5'7" x 8')|(5'x7'6")|(5'x8')|(6' x 7')|(6' x 7'5")|(60" x 90")|(60" x 96")|(64" x 100")|(72"x 84")|(7'7" x 5'3")|(8' x 5')|(oval 5' x 8')|(rectangle 5' x 8')|()'''
df['5x8'] = df['product_name'].apply(lambda x: re_extract(power, str(x)))

power=r'''(?i)(5' x 7(?!'\d))|(4'10" x 7')|(4'11" x 6'6")|(4'11" x 7')|(4'11" x 7'7")|(4'6" x 6'6")|(4'7" x 6'7")|(4'7" x 7'1")|(4'8" x 6'8")|(4'8" x 7'6")|(5' 3" x 7' 3")|(5' x 6' 7")|(5' x 6'6")|(5' x 7'(?!\d))|(5' x 7'1")|(5' x 7'2")|(5' x 7'3")|(5' x 7'4")|(5'1" x 6'6")|(5'1" x 7')|(5'2 x 7'3)|(5'2"  x 7'2")|(5'2" x 7')|(5'2" x 7'2")|(5'2" x 7'2")|(5'2" x 7'3")|(5'3 x 7'3)|(5'3" x 6'11")|(5'3" x 6'7")|(5'3" x 7')|(5'3" x 7'0")|(5'3" x 7'2")|(5'3" x 7'3")|(5'3" x 7'4")|(5'3" x 7'5")|(5'3"x7'3")|(5'6" x 6'10")|(5'x7'(?!6))|(5'x7'3")|(60" x 84")|(60"x 84")|(63" x 90")|(7'5" x 5'1")|(82\.67x59\.05x0\.31)|(83\.86x59\.84x1\.96)|(84x60x0\.39)|(90"l x 63"w x 0\.53"h)|(4'11"? x 7')|()'''
df['5x7'] = df['product_name'].apply(lambda x: re_extract(power, str(x)))

power=r'''(?i)(4' x 6')|(3' 6" x 5' 6")|(3' 6" x 6')|(3' 9" x 5' 5")|(3' 9" x 5' 7")|(3'10 x 5'10)|(3'10" x 5'10")|(3'10" x 5'5")|(3'10" x 5'8")|(3'10" x 6')|(3'10"x5'10")|(3'11 x 5'11)|(3'11 x 5'7)|(3'11" x 5')|(3'11" x 5'5")|(3'11" x 5'6")|(3'11" x 5'9")|(3'11" x 6')|(3'11" x 6'2")|(3'11"x5'11")|(3'5" x 5'5")|(3'6" x 5'6")|(3'6" x 5'9")|(3'6" x 6')|(3'6"x5'6")|(3'7" x 5'11")|(3'7" x 5'5")|(3'7" x 5'6")|(3'8" x 6')|(3'9 x 5'9)|(3'9" x 5')|(3'9" x 5'5")|(3'9" x 5'6")|(3'9" x 5'8")|(3'9" x 5'9")|(3'9"x5'9")|(4' 1" x 6')|(4' 3" x 6')|(4' 3" x 6' 3")|(4' 3" x 6' 6")|(4' 4" x 6)|(4' 5" x 6')|(4' 9" x 6')|(4' x 5')|(4' x 5' 6")|(4' x 5' 7")|(4' x 5'10")|(4' x 5'11")|(4' x 5'4")|(4' x 5'5")|(4' x 5'6")|(4' x 5'7")|(4' x 5'8")|(4' x 5'9")|(4 x 6)|(4' x 6')|(4' x 6' oval)|(4' x 6'2")|(4' x 6'6")|(4'1" x 5'7")|(4'1" x 6')|(4'3" x 5')|(4'3" x 5'11")|(4'3" x 6')|(4'3" x 6'3")|(4'3" x 6'6")|(4'4" x 5'11")|(4'4" x 6')|(4'4" x 6'2")|(4'4" x 6'3")|(4'4" x 6'5")|(4'4" x 6'6")|(4'5" x 6'5")|(4'6" x 6')|(46" x 66")|(48" x 72")|(48" x72")|(4'x6')|(60" x 48")|(60"l x 51"w x 0\.25"h)|(70\.86x47\.24x0\.31)|(72\.04x48\.03x0\.39)|(72\.05x48\.03x1\.96)|(oval 4' x 6')|(3'9.?" x 5'9.?")|()'''
df['4x6'] = df['product_name'].apply(lambda x: re_extract(power, str(x)))

power=r'''(?i)(3' x 5'(?! Oval))|(1\.5"dx4 ft"wx3 ft"h)|(2'10" x 5')|(2'11" x 4'11")|(2'11" x 4'3")|(2'5" x 4'5")|(2'6" x 4')|(2'6" x 4'0")|(2'6" x 4'9")|(2'6" x 5')|(2'6"x 4')|(2'7" x 3'10")|(2'7" x 3'9")|(2'7" x 4'11")|(2'7" x 4'7")|(27" x 48")|(2'7" x 5')|(2'7" x 5'11")|(2'7" x 5'3")|(2'8" x 4'11")|(2'8" x 4'2")|(2'9" x 4'9")|(2'9" x 5')|(3' 10" x 5')|(3' 5" x 5')|(3'.?6.?" x 5')|(3' x 4')|(3' x 4\.75')|(3' x 4'11")|(3' x 4'2")|(3' x 4'6")|(3' x 4'9")|(3 x 5)|(3' x 5')|(3' x 5' oval)|(3' x 5'1")|(3' x 5'5")|(3' x 57")|(30" x 48")|(30" x 50")|(3'1" x 4'7")|(3'1" x 5')|(3'1" x 5'1")|(3'1" x 5'3")|(3'10" x 5')|(3'10" x 5'1")|(3'10" x 5'2")|(3'10" x 5'4")|(3'10" x 5'7")|(3'11 x 6'2)|(3'11" x 5'10")|(3'11" x 5'11")|(3'11" x 5'2")|(3'11" x 5'3")|(3'11" x 5'4")|(3'11" x 5'7")|(3'11" x 6'7")|(3'2" x 4'6")|(3'2" x 5')|(3'2" x 5'4")|(32" x 56")|(3'2" x 5'7")|(3'3" x 4'11")|(3'3" x 4'10")|(3'3" x 4'11")|(3'3" x 4'7")|(3'3" x 5')|(3'3" x 5'1")|(3'3" x 5'2")|(3'3" x 5'3")|(3'3" x 5'4")|(3'3" x 5'5")|(3'3" x 5'6")|(3'3" x 5'7")|(3'4" x 4'7")|(3'4" x 5')|(3'4" x 5'4")|(34\.5" x 58")|(3'5" x 5'2")|(35" x 57")|(35" x 59")|(3'6" x 4'4")|(36" x 48")|(3'6" x 5'2")|(3'6" x 5'3")|(36" x 54")|(36" x 55")|(36" x 59")|(36" x 60")|(3'7" x 5'1")|(3'7" x 5'2")|(3'7" x 5'7")|(3'9" x 5'2")|(3'9" x 5'4")|(39" x 58")|(39" x 59")|(3'x 5')|(4' x 5'3")|(40 in l x 62 in w)|(40" x 54")|(40" x 60")|(42" x 66")|(45" x 69")|(46" x 57")|(48" x 60")|(50"l x 31"w x 0\.47"h)|(50"l x 31"w x 0\.53"h)|(oval 3' x 5')|(rectangle 3' x 5')|()'''
df['3x5'] = df['product_name'].apply(lambda x: re_extract(power, str(x)))

power=r'''(?i)(2' x 3'(?! round))|(1'10" x 2'10")|(1'10" x 2'6")|(1'10" x 2'7")|(1'10" x 2'8")|(1'10" x 3')|(1'10" x 3'2")|(1'10" x 3'3")|(1'10"x2'10")|(1'11" x 2'11")|(1'11" x 3')|(1'11" x 3'7")|(1'11" x 3'9")|(12" x 36")|(1'5" x 2'10")|(1'6" x 2'5")|(1'6" x 2'6")|(1'6" x 3')|(1'7" x 2'5")|(1'7" x 2'6")|(1'7" x 2'7")|(1'7" x 2'8")|(1'8" x 2'6")|(1'8" x 2'10")|(1'8" x 2'6")|(1'8" x 2'7")|(1'8" x 2'7\.5")|(1'8" x 2'8")|(1'8" x 2'9")|(18" x 30")|(1'8" x 3'4")|(18" x 36")|(1'8" x 3'7")|(1'8" x 3'8")|(18" x 48")|(18"x29")|(1'9" x 2'10")|(1'9" x 2'9")|(1'9" x 3')|(19" x 30")|(19" x 32")|(19\.5" x 47")|(19\.6" x 31\.5")|(19\.6" x 35\.4")|(19\.7" x 31\.5")|(2' 6" x 4')|(2' x 2'11")|(2' x 2'7")|(2' x 2'9")|(2 x 3)|(2' x 3'(?! round))|(2' x 3' 7")|(2' x 3'11")|(2' x 3'3")|(2' x 3'4")|(2' x 3'4")|(2' x 3'6")|(2' x 3'7")|(2' x 3'9")|(20 in l x 32 in w)|(20" x 30")|(20" x 32")|(20" x 33")|(20" x 34")|(20" x 35")|(20" x 36")|(20" x 39")|(20" x 48")|(20" x 54")|(20"x30")|(20\.5" x 32\.5")|(21" x 31")|(21" x 34")|(2'1" x 3'8")|(2'11" x 3'3")|(2'2 x 3'9)|(2'2" x 3')|(22" x 31")|(2'2" x 3'11")|(2'2" x 3'2")|(2'2" x 3'3")|(2'2" x 3'4")|(22" x 34")|(22" x 35")|(22" x 36")|(2'2" x 3'8")|(2'2" x 3'9")|(22" x 40")|(22" x 44")|(22"x34")|(2'2"x3'9")|(2'3" x 3')|(2'3" x 3'10")|(2'3" x 3'11")|(2'3" x 3'3")|(23" x 35")|(23" x 36")|(2'3" x 3'6")|(2'3" x 3'6")|(2'3" x 3'7")|(2'3" x 3'7")|(2'3" x 3'8")|(2'3" x 3'9")|(2'3" x 4')|(23" x 45")|(24 in l x 35 in w)|(24 in l x 36 in w)|(2'4" x 3'10")|(24" x 36")|(2'4" x 3'6")|(2'4" x 3'7")|(24" x 37\.5")|(24" x 39")|(2'4" x 3'9")|(2'4" x 4')|(24" x 40")|(2'4" x 4'3")|(2'4" x 4'8")|(24"x39")|(25" x 35")|(2'5" x 3'8")|(2'5" x 3'9")|(2'5" x 4'1")|(2'6" x  4')|(2'6" x 3')|(2'6" x 3'10")|(2'6" x 3'11")|(2'6" x 3'6")|(2'6" x 3'9")|(2'6" x 4')|(2'6" x 4'2")|(2'6" x 4'3")|(26" x 45")|(2'7" x 3'11")|(27" x 36")|(2'7" x 3'8")|(2'7" x 4')|(2'7" x 4'1")|(27" x 42")|(2'7" x 4'2")|(2'7" x 4'3")|(27" x 45")|(27\.5" x 43\.3")|(27\.6" x 43\.3")|(2'8" x 4')|(29" x 46")|(29\.5" x 47")|(30" x 18")|(30" x 20")|(30" x 36")|(30" x 40")|(30" x 45")|(30" x 46")|(30" x 47")|(30"x20")|(31" x 45")|(32 in l x 48 in w)|(32" x 18")|(32" x 20")|(32"l x 22"w x 0.47"h)|(32"l x 22"w x 0.53"h)|(34" x 21")|(35" x 22")|(35" x 46")|(35" x 47")|(36" x 23")|(36"d x 23"w x 0.39"h)|(36"l x 23"w x 0.79"h)|(36.22x24.01x0.39)|(36.22x24.02x1.96)|(38" x 32")|(47" x 35")|(48" x 30")|(48" x 36")|(1'9.?" x 2'9.?")|(2' x 3(?!' round))|()'''
df['2x3'] = df['product_name'].apply(lambda x: re_extract(power, str(x)))

In [14]:
print(len(df))
# df[df['product_name'].astype(str)=='''Momeni 2'3\" X 7'6\" Runner Baja Rug in Denim''']
df['product_name'].explode().value_counts()[0:500]
# df[500:1000]
print(len(df[df['7Runner'].astype(str)!='[]']))
df[df['7Runner'].astype(str)!='[]']

2829
360


buckets external_id  \
50    Rugs & Mats B    43436828   
96    Rugs & Mats B    61945869   
98    Rugs & Mats B    61945791   
100   Rugs & Mats B    61945654   
101   Rugs & Mats B    61945609   
102   Rugs & Mats B    61945449   
103   Rugs & Mats B    61943308   
105   Rugs & Mats B    61943162   
106   Rugs & Mats B    61943193   
107   Rugs & Mats B    62390354   
108   Rugs & Mats B    62390231   
111   Rugs & Mats B    62641586   
112   Rugs & Mats B    62641630   
113   Rugs & Mats B    62641531   
114   Rugs & Mats B    62641388   
115   Rugs & Mats B    62641272   
116   Rugs & Mats B    62641326   
117   Rugs & Mats B    62641227   
118   Rugs & Mats B    62641029   
120   Rugs & Mats B    62640930   
121   Rugs & Mats B    62640688   
123   Rugs & Mats B    62724432   
124   Rugs & Mats B    62724470   
128   Rugs & Mats B    63182491   
142   Rugs & Mats B    63892383   
143   Rugs & Mats B    63891720   
144   Rugs & Mats B    63891669   
145   Rugs & Mats B    63891904   
146   Rugs & Mats B    63891966   
147   Rugs & Mats B    63891843   
148   Rugs & Mats B    63891782   
149   Rugs & Mats B    63892024   
338   Rugs & Mats B    69450143   
342   Rugs & Mats B    69473833   
349   Rugs & Mats B    69495796   
350   Rugs & Mats B    69495800   
351   Rugs & Mats B    69495808   
352   Rugs & Mats B    69495812   
406   Rugs & Mats B    61945104   
407   Rugs & Mats B    62641685   
408   Rugs & Mats B    62641487   
409   Rugs & Mats B    62641173   
410   Rugs & Mats B    62641128   
411   Rugs & Mats B    62641074   
412   Rugs & Mats B    62640985   
413   Rugs & Mats B    62640886   
418   Rugs & Mats B    62724395   
419   Rugs & Mats B    63182170   
420   Rugs & Mats B    63182354   
421   Rugs & Mats B    63182422   
422   Rugs & Mats B    63182286   
430   Rugs & Mats B    63891546   
545   Rugs & Mats B    68372477   
557   Rugs & Mats B    69448706   
567   Rugs & Mats B    69495780   
568   Rugs & Mats B    69495788   
569   Rugs & Mats B    69495792   
582   Rugs & Mats B    69586257   
583   Rugs & Mats B    69586261   
584   Rugs & Mats B    69586260   
585   Rugs & Mats B    69586262   
586   Rugs & Mats B    69586263   
587   Rugs & Mats B    69586270   
588   Rugs & Mats B    69586271   
589   Rugs & Mats B    69585895   
590   Rugs & Mats B    69585901   
612   Rugs & Mats B    69648688   
613   Rugs & Mats B    69648694   
614   Rugs & Mats B    69648712   
615   Rugs & Mats B    69648730   
616   Rugs & Mats B    69648754   
617   Rugs & Mats B    69648766   
623   Rugs & Mats B    69650641   
624   Rugs & Mats B    69650655   
625   Rugs & Mats B    69650662   
626   Rugs & Mats B    69650669   
627   Rugs & Mats B    69650676   
633   Rugs & Mats B    61945364   
636   Rugs & Mats B    62390316   
637   Rugs & Mats B    62390279   
640   Rugs & Mats B    62640732   
711   Rugs & Mats B    43437030   
760   Rugs & Mats B    69694821   
761   Rugs & Mats B    69694827   
762   Rugs & Mats B    69694845   
789     Rugs & Mats    63182705   
900     Rugs & Mats    77331280   
986     Rugs & Mats    75926327   
987     Rugs & Mats    69160370   
1041    Rugs & Mats    43437061   
1049    Rugs & Mats    62641739   
1200    Rugs & Mats    75926335   
1231    Rugs & Mats    61279353   
1302    Rugs & Mats    69648718   
1303    Rugs & Mats    69648736   
1310    Rugs & Mats    63182637   
1356    Rugs & Mats    69156267   
1357    Rugs & Mats    69154119   
1358    Rugs & Mats    69154171   
1359    Rugs & Mats    69158971   
1360    Rugs & Mats    69160257   
1361    Rugs & Mats    69160899   
1368    Rugs & Mats    75926470   
1372    Rugs & Mats    69586259   
1393    Rugs & Mats    62724357   
1451    Rugs & Mats    69160837   
1470    Rugs & Mats    69694833   
1471    Rugs & Mats    69694839   
1472    Rugs & Mats    69694857   
1526    Rugs & Mats    69648748   
1527    Rugs & Mats    69650091   
1539    Rugs & Mats    69648742   
1572    Rugs & Mats    69716701   
1573    Rugs & Mats   

In [6]:
err

NameError: name 'err' is not defined

In [ ]:
print('Certifications:')
df['buckets'].explode().value_counts().reset_index()['index'].to_list()[0:50]

In [ ]:
power=r'(?i)(AHAM)|(ANSI)|(ASTM)|(CADR)|(\bCARB\b)|(ce.?marketing)|(certi.?pur.?us)|(CDPH.?01350)|(csa.?listed)|(dlc.?approved)|(energy.?star)|(fair.?trade.?certified)|(fcc.?cert)|(fda.?appro)|(greenguard)|(JPMA)|(ASME)|(CPSC)|(CSA)|(UPC)|(met.?lab)|(met.?listed)|(non.?gmo)|(oeko.?tex)|(osha)|(rohs)|(true.?air)|(\b(?<!\<)ul(?!\>)\b)|(usda.?certified)|(zero.?voc)|(\bETL\b)|()'
df['power'] = df['LONG_DESCRIPTION'].apply(lambda x: re_extract(power, str(x)))
df[df['power'].astype(str)!='[]']

# Width

In [ ]:
df_width= df[(df['width'].astype(str)!='[]')]
df_width['width']=df_width['width'].apply(lambda x: natsorted(x))
df_width['width_one']=df_width['width'].apply(lambda x: re.sub(r"",'',str(x)))
clean_data(df_width,'width_one','width_two','width_three')

df_width['width_three']=df_width['width_three'].str[2:-2].str.split('","').apply(lambda x: '["'+'","'.join(natsorted(x))+'"]')
rounding(df_width,'width_three','a-eghj-mo-su-z')
df_width=df_width[df_width['rounding'].astype(str)=='[]']
print(len(df_width))
curate(df_width, 'width_three', formatted_attribute)
matchwidth=df_width[['external_id',curation_col]]
rounding(df_width, curation_col,'a-eghj-mo-su-z')

# Diameter

In [ ]:
df_diam=df[(df['width'].astype(str)=='[]')&(df['diameter'].astype(str)!='[]')]
df_diam['height']=df_diam['diameter'].apply(lambda x: natsorted(x))
df_diam['height_final']=df_diam['height'].apply(lambda x: re.sub(r'''(?i)(\\\"\s?diameter)|(\\?\"?\”?(?:inc?h?)?(?<!').(diameter|dia))|(\\?('')\W?(diameter|dia))|(\\ in)|(square\W?Inch)''',' in',str(x))).apply(lambda x: re.sub(r'''(\\?(?:(?<!')'(?!'))?(?:f?t?)?(?:f?o?o?t?)?\W?(diameter|dia))|(square\W?foot)|(\'\\xa.{0,2}diameter)''',' ft',str(x))).apply(lambda x: re.sub(r'''(?i)(\\ in)''',' in',str(x))).apply(lambda x: re.sub(r'''\[|\]|'|\\|"''','',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))

curate(df_diam, 'height_final', formatted_attribute)
df_diam[curation_col]=df_diam[curation_col].apply(lambda x: re.sub(r'mter','',str(x)))
rounding(df_diam, curation_col,'a-eghj-mo-su-z')
df_diam=df_diam[df_diam['rounding'].astype(str)=='[]']
matches_diam=df_diam[['external_id',curation_col]]
print(len(matches_diam))

# More

In [ ]:
df_more=df[(df['width'].astype(str)=='[]')&(df['blank_matches_long'].astype(str)=='[]')&(df['blank_matches'].astype(str)=='[]')&(df['width'].astype(str)!='[]')]
print(len(df_more))
# df_more

In [ ]:
def frac2string(s):
    i, f = s.groups(0)
    f = round_string_float(Fraction(f))
    return str(int(i) + round_string_float(float(f)))

import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A

def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()

In [ ]:
df_more['matches_more']=df_more['width'].apply(lambda x: re.sub(r'''\\" W|".?W|''.?W|”.?W|˝.?W|”.?W|” w|″ W''',' in',str(x))).apply(lambda x: re.sub(r'''\\' W|'.?W|'.?W''',' ft',str(x))).apply(lambda x: re.sub(r'\\" W|"W|\"xa0W|xa0W|"','',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))
df_more['matches_more']=df_more['matches_more'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])

In [ ]:
df_more['matches_more']=df_more['matches_more'].apply(lambda x: re.sub("'",'"',str(x))).apply(lambda x: re.sub(",",'","',str(x))).apply(lambda x: re.sub('" ','"',str(x))).apply(lambda x: re.sub('","',' in","',str(x))).apply(lambda x: re.sub('"]',' in"]',str(x))).apply(lambda x: re.sub('in in','in',str(x))).apply(lambda x: re.sub('ft in','ft',str(x)))                                                                           
df_more['matches_more']=df_more['matches_more'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(str(x))]))
df_more['matches_more']=df_more['matches_more'].apply(lambda x: natsorted(ast.literal_eval(re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))))
df_more['matches_more']=df_more['matches_more'].apply(lambda x: re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))

df_more[curation_col]=df_more['matches_more']
matchesmore=df_more[['external_id',curation_col]]
print(len(matchesmore))

In [ ]:
df_na=df[(df['width'].astype(str)=='[]')&(df['length'].astype(str)=='[]')&(df['height'].astype(str)=='[]')&(df['width'].astype(str)=='[]')&(df['diameter'].astype(str)=='[]')&(df['blank_matches'].astype(str)=='[]')&(df['blank_matches_long'].astype(str)=='[]')]#&(df['new_matches'].astype(str)!='[]')]
df_na[curation_col]='n/a'
matchesnadf=df_na[['external_id',curation_col]]
print(len(matchesnadf))

# Two Min Max (Min is width)

In [ ]:
dubs['dub']=dubs['double_long'].apply(lambda x: re.sub(r'''(?i)(\-in)|(\s?\\{0,3}\s?"\s?|(?<!\d)\s(?!\d))|([a-wyz])|(”)|(\[)|(\])|(\\{0,2}xa?0(?!\.))''','',str(x))).apply(lambda x: f'"[{x}]"').astype(str)
df1 = dubs['dub'].str[2:-2].str.split(',').explode().str.split(r'(?i)x', expand=True).fillna('')
df1[1]=df1[1].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, str(x))]).apply(lambda x: re.sub(r'''('?\[(?:"|')\[(?:"|'))''','["',str(x))).apply(lambda x: re.sub(r'''('\](?:"|')\])|(?:\[|\]|"|'|\\+)''','',str(x)))
df1[0]=df1[0].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, str(x))]).apply(lambda x: re.sub(r'''('?\[(?:"|')\[(?:"|'))''','["',str(x))).apply(lambda x: re.sub(r'''('\](?:"|')\])|(?:\[|\]|"|'|\\+)''','',str(x)))

x=df1[1].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)]).to_list()
y=df1[0].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)]).to_list()

dubs[['Max', 'Min']] = pd.concat([df1.astype(float).max(1).groupby(level=0).apply(list), 
                                df1.astype(float).min(1).groupby(level=0).apply(list)], axis=1)

clean_data(dubs,'Min','doubles','doub')
dubs['doubs']=dubs['doub'].apply(lambda x: re.sub(r'\[\s?\"\s?\[\s?(:"|")?','',str(x))).apply(lambda x: re.sub(r'"?\]"\]','"]',str(x)))
curate(dubs,'doubs','length')
rounding(dubs,curation_col,'a-eghj-mo-su-z')

In [ ]:
pat_split=r'''(?i)(\d+(?:(?:\-?\s?\d+\/\d+)?(?:\.\d+)?)?\\?(?:(?:")?(?:”)?(?:'{0,2})?(?:\Winc?h?e?s?)?(?:\Wft)?(?:\Wfoot)?).?x.?\d+(?:(?:\-?\s?\d+\/\d+)?(?:\.\d+)?)?\\?(?:(?:")?(?:”)?(?:'{0,2})?(?:\Winc?h?e?s?)?(?:\Wft)?(?:\Wfoot)?))|()'''
regex_pattern_blank=r'(?i)(\d+.?ft)|(\d+.?foot)|(\d+.?feet)|(\d+.?inch)|(\d+.?")|(\d+")|()'
pat_comma=r"\,"

df['matches_split'] = df['product_name'].apply(lambda x: re_extract(pat_split, str(x)))
df['comma']=df['matches_split'].apply(lambda x: re_extract(pat_comma, str(x)))
df['matchez'] = df['product_name'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))

split=df[(df['matches_split'].astype(str)!='[]')&(df['length'].astype(str)=='[]')&(df['width'].astype(str)=='[]')]
split=split[split['comma'].astype(str)=='[]']
print(len(split))

In [ ]:
split['matches_split']=split['matches_split'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, str(x))])
split['matches_split']=split['matches_split'].str.get(0).apply(lambda x: x.lower())
split=split[split['matches_split'].astype(str)!="['13 x 13 inches', '6.5 x 6.5 inches']"]
split[['length', 'widths']]=split['matches_split'].str.split("x",expand=True,)
print(len(split))

In [ ]:
split['length']=split['length'].apply(lambda x: re.sub(r"\[\'",'',str(x))).apply(lambda x: re.sub(r"(?i)\W?feet|\W?foot|\\?\\?\'",'ft',str(x))).apply(lambda x: re.sub(r"ft\s?ft",'ft',str(x))).apply(lambda x: re.sub(r'-inch|-inc|\\?\\?\"',' in',str(x))).apply(lambda x: re.sub(r"\['|']|8x/1x|4x4 b|26x31 g|8x/1",'',str(x))).apply(lambda x: re.sub(r"ft",' ft',str(x))).apply(lambda x: re.sub(r"2 24 in",'24 in',str(x))).apply(lambda x: re.sub(r"33 (?!in)",'33 in',str(x))).apply(lambda x: re.sub(r" ",'',str(x))).apply(lambda x: re.sub(r"in",' in',str(x))).apply(lambda x: re.sub(r"5 inch",'31.5 in',str(x))).apply(lambda x: re.sub(r"20x24','20",'20 in',str(x)))                        
split['widths']=split['widths'].apply(lambda x: re.sub(r"\'\]",'',str(x))).apply(lambda x: re.sub(r"(?i)\W?feet|\W?foot|\\?\\?\'",'ft',str(x))).apply(lambda x: re.sub(r"ft\s?ft",'ft',str(x))).apply(lambda x: re.sub(r'-inch|-inc|\\?\\?\"',' in',str(x))).apply(lambda x: re.sub(r"\['|']|None| |4-f",'',str(x))).apply(lambda x: re.sub(r"in",' in',str(x))).apply(lambda x: re.sub(r"ft",' ft',str(x))).apply(lambda x: re.sub(r"24 in','20x24 in",'24 in',str(x))).apply(lambda x: re.sub(r"27b",'27 in',str(x))).apply(lambda x: re.sub(r"24(?! )",'24 in',str(x))).apply(lambda x: re.sub(r'\[|\]','',str(x)))

pat_split=r'''(?i)(ft)|()'''
split['ft'] = split['widths'].apply(lambda x: re_extract(pat_split, str(x)))
split['ft_length'] = split['length'].apply(lambda x: re_extract(pat_split, str(x)))
print(len(split))

In [ ]:
split=split[split['length'].astype(str)!=""]
split=split[split['widths'].astype(str)!=""]
split=split[split['comma'].astype(str)=='[]']
split_inch=split[(split['ft'].astype(str)=='[]')&(split['ft_length'].astype(str)=='[]')]

split_inch['length']=split_inch['length'].apply(lambda x:re.sub(r'''\s?in|\s?inche?s?|ch|\(|\,|es|\['|'\]|\-|\?|\/''','',str(x)))
split_inch['widths']=split_inch['widths'].apply(lambda x:re.sub(r'''\s?in|\s?inche?s?|ch|\(|\,|es|\['|'\]|\-|\?|\/''','',str(x))).apply(lambda x: re.sub(r'','',str(x)))
split_inch['length']=split_inch['length'].astype(float)
split_inch['widths']=split_inch['widths'].astype(float)
split_inch['Max'] = split_inch[['length', 'widths']].values.max(1)
split_inch['Min'] = split_inch[['length', 'widths']].values.min(1)
split_inch['Min']=split_inch['Min'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))
print(len(split_inch))
split_inch[curation_col]=split_inch['Min']
split_inch[curation_col]=split_inch[curation_col].apply(lambda x: str(f"['{x} in']")).apply(lambda x: re.sub(r"'",'"',str(x)))
rounding(split_inch, curation_col,'a-eghj-mo-su-z')

In [ ]:
msplit_inch=split_inch[['external_id',curation_col]]

# N/A values

In [ ]:
print(len(df[(df['matchez'].astype(str)=='[]')&(df['width'].astype(str)=='[]')&(df['matches_split'].astype(str)=='[]')&(df['diameter'].astype(str)=='[]')]))
na_values=df[(df['matchez'].astype(str)=='[]')&(df['width'].astype(str)=='[]')&(df['matches_split'].astype(str)=='[]')&(df['diameter'].astype(str)=='[]')]
na_values[curation_col]='n/a'
matches_na_values=na_values[['external_id',curation_col]]

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchesnadf))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matches_na_values))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matches_diam))

In [ ]:
print(len(df))
print(len(matchesnadf)+len(matches_na_values)+len(matches_diam)+len(matchwidth))#+len(msplit_inch)) #len(msplit_ft)+

In [ ]:
print(len(df[(df['width'].astype(str)=='[]')&(df['matches_split'].astype(str)=='[]')&(df['diameter'].astype(str)=='[]')&(df['width'].astype(str)=='[]')&(df['blank_matches'].astype(str)=='[]')]))
# df[(df['width'].astype(str)=='[]')&(df['matches_split'].astype(str)=='[]')&(df['diameter'].astype(str)=='[]')&(df['matches_all'].astype(str)=='[]')&(df['blank_matches'].astype(str)=='[]')]


In [ ]:
rounding(split_inch,curation_col,'a-eghj-mo-su-z')

In [ ]:
rounding(df_diam, curation_col,'a-eghj-mo-su-z')

In [ ]:
stop   

# send to the folder for upload

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}_{get_df_name(matches)}-{today}_matches.csv',index=False) 
looks_good('Bed Bath & Beyond', attribute, matchesnadf,today)

In [ ]:
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# def looks_good(customer, attribute, matches,today): 
#     drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     matches.to_csv(f'{drive_path}/BBBY - {attribute}_{get_df_name(matches)}-{today}_matches.csv',index=False) 
# looks_good('Bed Bath & Beyond', attribute, msplit_inch,today)

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, attribute, matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}_{get_df_name(matches)}-{today}_matches.csv',index=False) 
looks_good('Bed Bath & Beyond', attribute, matches_na_values,today)

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, attribute, matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}_{get_df_name(matches)}-{today}_matches.csv',index=False) 
looks_good('Bed Bath & Beyond', attribute,matches_diam,today)

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, attribute, matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}_{get_df_name(matches)}-{today}_matches.csv',index=False) 
looks_good('Bed Bath & Beyond', attribute,matchwidth,today)